### Пропажа и поиски объектов в пространстве


##### Функция построения гистограммы

на вход принимает данные, интересующий показатель, кол-во интервалов и алгоритм классификации


In [ ]:
def plot_hist_with_intervals(ax, data, column, scheme, k=None):
    classification_schemes = {
        "equalInterval": mc.EqualInterval,
        "naturalBreaks": mc.NaturalBreaks,
        "quantiles": mc.Quantiles,
        "stdMean": mc.StdMean,
        "percentiles": mc.Percentiles,
        "userDefined": mc.UserDefined,
        "boxPlot": mc.BoxPlot
    }
    
    classification_class = classification_schemes.get(scheme)

    if classification_class:
        # Проверяем, нужно ли учитывать параметр k для данной схемы
        if scheme in ["stdMean", "percentiles", "boxPlot"]:
            classifier = classification_class(data[column])
        else:
            classifier = classification_class(data[column], k=k)
        
        # Строим гистограмму
        ax.hist(data[column], bins=20)
        
        # Добавляем вертикальные линии для границ интервалов
        for val in classifier.bins:
            ax.axvline(x=val, color='red', linestyle='--')
        
        # Добавляем подписи и заголовок
        ax.set_xlabel('Value')
        ax.set_ylabel('Frequency')
        ax.set_title('Histogram with Interval Boundaries')
    else:
        print(f"Схема классификации '{scheme}' не найдена.")


##### Функция для создания карты

на вход принимает данные, кол-во интервалов и метод классификации


In [ ]:
def plot_classified_map(ax, data, column, scheme, k=None, cmap='BuPu', title='Population Density, ppl/sq.km'):
    # Строим график
    data.plot(column=column, cmap=cmap, linewidth=0.8, edgecolor='0.8', legend=True, scheme=scheme, k=k, ax=ax)

    # Добавляем название
    ax.set_title(title)

    # Скрываем координатные оси
    ax.axis('off')

    # Получаем объект легенды
    leg = ax.get_legend()

    # Перемещаем легенду за пределы графика справа
    leg.set_bbox_to_anchor((1.5, 1))



Если не разобраться с тем, как устроены системы координат, то ваши данные могут загадочно перемещаться в пространстве. Например:

1. набор данных внезапно оказывается в Гвинейском заливе

2. объекты Москвы перемещаются в кенийскую деревню, но там они во много раз меньше, чем должны быть (часто, их можно отыскать в деревне Рамула, недалеко от озера Виктория - вот [тут](https://maps.app.goo.gl/R6LrXgzz1qniSU2b6), думаю Рамула должна стать еще одиним городом-побратимом (Sister city) Москвы!)

Это часто происходит, когда путают перепроецирование с установкой системы координат

Задача этого блока сделать так, чтобы как можно меньшее количество их вас в дальнейшнем сталкивались с такими проблемами :)


### Перепроецирование VS установка системы координат


Во время перепроецирования происходит пересчет координат из одной системы координат в другую. У нас не только меняется СК у набора данных, но и перезаписываются координаты для всех объектов.

Если мы устанавливаем систему координат, то в этот момент пересчет координат не происходит, у нас просто перезаписывается определение системы координат у данных.

Если мы сопоставим Перепроецирование VS установка системы координат

Представим, что у нас есть 100 евро. Мы их хотим поменять на рубли.


In [ ]:
admin_okrug.to_crs(data_crs)

In [ ]:
admin_okrug.set_crs(data_crs)

Пользоваться set_crs можно только в том случае, если проекция у данных не установлена или установлена неверно


In [12]:
import requests

# Set up GraphHopper API endpoint
endpoint = "https://graphhopper.com/api/1/route"

# Coordinates for Vienna and Berlin
vienna_coords = "48.206389,16.384722"  # Vienna
berlin_coords = "52.5251, 13.3694" 



# Parameters for the request
params = {
    "key": "f879a2e8-af71-4e73-956e-6f520a03722a",
    "point": [vienna_coords, berlin_coords],
    "profile": "train",
    # "locale": "en",  # Language for response
}

# Send the request
response = requests.get(endpoint, params=params)

# Parse the response
if response.status_code == 200:
    route_data = response.json()
    # Extract and process the route data as needed
    print(route_data)
else:
    print("Error:", response.status_code)


Error: 400


In [7]:
route_data

{'hints': {'visited_nodes.sum': 2702, 'visited_nodes.average': 2702.0},
 'info': {'copyrights': ['GraphHopper', 'OpenStreetMap contributors'],
  'took': 16,
  'road_data_time_stamp': '2024-05-16T15:00:00Z'},
 'paths': [{'distance': 682000.364,
   'weight': 34312.895034,
   'time': 24028509,
   'transfers': 0,
   'points_encoded': True,
   'points_encoded_multiplier': 100000.0,
   'bbox': [13.385715, 48.191335, 16.690781, 52.520001],
   'points': '}qfeHs~|bBLa@B?H?RHLJVLFFm@~BxBr@XiBXaAj@oAl@g@~@cBJk@l@sANQh@}@h@iAxBmGPo@`FfFj@eB`C}GNWrBiGH_@UQiBgBgCuDwDyFcAoBKWE[J{A?YGi@ASSI{F_El@uCl@}Bp@cBjBaDz@kAp@w@pAeBjAyAz@u@x@w@lAgBNYfA_BC_@q@_A_@a@_B{BVkEZoEPeAHYL[lAcCgCuCoCyCOUlAwBt@mBX{@\\{A^_Dp@sGr@}Fj@qEd@gD^_BRq@b@eAd@{@l@_Af@i@z@s@pBqA`@c@dDeC|AoA~EmEpB{B^g@r@eAh@s@nA}Ad@s@x@yAzAyCXMZ]RS`@WZOTWPULYr@}C`@uALYz@aBz@_BZYJIJCZATDJFHFRXLb@D`@@NAVARMd@ENKPKJKFUFU?UE}AcAeEgH}FqJaAuAQ_@iFiIwZsg@mIwNk@y@cE_HuBqDaBkCw@sAo@_A{So]aCaEg@s@mBaDU[Wa@eGeKcE}GSSaBeCy@aAw@y@o@i@m@_@{@c@m@Sk@My@Ms@Ek@AgCBkCD

In [19]:
import requests

# HERE Routing API endpoint
routing_endpoint = "https://intermodal.router.hereapi.com/v8/routes"

# HERE API Key (replace 'YOUR_API_KEY' with your actual API key)
api_key = "UdxF47WFsWjfHYOX3-9AE2VHtV_a2qg4XhYIia2jAv8"

# Start and end coordinates (latitude, longitude)
start_point = "48.206389,16.384722"  # Berlin, Germany
end_point = "52.5251, 13.3694"       # Paris, France

# Request parameters
params = {
    "origin": start_point,
    "destination": end_point,
    "apiKey": api_key,
    "transportMode": "train",  # Specify the mode of transport (e.g., car, pedestrian, publicTransport)
    "return": "summary"      # Return only summary information about the route
}

# Send GET request to HERE Routing API
response = requests.get(routing_endpoint, params=params)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the JSON response
    data = response.json()
    
    # Extract route summary information
    distance = data['routes'][0]['sections'][0]['summary']['length']  # Distance in meters
    duration = data['routes'][0]['sections'][0]['summary']['duration']  # Duration in seconds
    
    print(f"Route from Berlin to Paris:")
    print(f"Distance: {distance / 1000:.2f} km")
    print(f"Duration: {duration / 60:.2f} minutes")
else:
    print("Error:", response.status_code)


Error: 400
